In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Carga de los datos de entrenamiento y validacion

In [3]:
train_dir = '../../dataset/entrenamiento'
test_dir = '../../dataset/test'

datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normaliza los valores de píxeles al rango [0, 1]
    validation_split=0.25  # Porcentaje de datos para validación
)

batch_size = 32  # Tamaño de lote.
image_size = (100, 100)  # tamaño de las imagenes para ser ajustada.

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Para clasificación con 5 clases
    subset='training'
)

validation_data = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 14528 images belonging to 16 classes.
Found 4832 images belonging to 16 classes.


In [3]:
total_data_train = train_data.samples
total_data_validation = validation_data.samples

print("Total de datos de entrenamiento:", total_data_train)
print("Total de datos de validación:", total_data_validation)

total_clases = train_data.num_classes

print("Total de clases:", total_clases)

Total de datos de entrenamiento: 14528
Total de datos de validación: 4832
Total de clases: 16


## Implementacion de los modelos

In [52]:
input_shape = (100, 100, 3)
# Aplicacion de la misma cantidad de capas convolucionales que densas.
modeloV1 = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(total_clases, activation='softmax')
])

# Aplicacion de mas capas densas que convolucionales.
modeloV2 = tf.keras.models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(total_clases, activation='softmax')
])
# Aplicacion de mas capas convolucionales que densas
modeloV3 = tf.keras.models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(total_clases, activation='softmax')
])

# Aplicacion de misma cantidad de capaz aumentando Dropout.
modeloV4 = tf.keras.models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(total_clases, activation='softmax')
])

## Implementacion de modelos con modelos pre entrenados

In [ ]:
import tensorflow_hub as hub

# Capa Keras de TensorFlow Hub con tamaño de entrada fijo
url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
mobilenetv2 = hub.KerasLayer(url, input_shape=input_shape)

# Congelar el modelo descargado
mobilenetv2.trainable = False

# Modelo secuencial que acepta tamaño de entrada variable
modeloMobileNet = tf.keras.Sequential([
    mobilenetv2,
    layers.Dense(total_clases, activation='softmax')
])


In [53]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Cargar el modelo preentrenado InceptionV3 sin la capa superior 
# (incluyendo pesos)
in_base_model = InceptionV3(weights='imagenet', 
                            include_top=False, 
                            input_shape=input_shape)

# Agregar capas personalizadas para la clasificación
in_x = in_base_model.output
in_x = GlobalAveragePooling2D()(in_x)
in_x = Dense(1024, activation='relu')(in_x)
in_predictions = Dense(total_clases, activation='softmax')(in_x)

# Crear el modelo completo
modeloImageNet = Model(inputs=in_base_model.input, outputs=in_predictions)

# Congelar las capas del modelo base (InceptionV3) 
# para no entrenarlas nuevamente
for layer in in_base_model.layers:
    layer.trainable = False

In [54]:
from tensorflow.keras.applications import DenseNet121

# Cargar el modelo preentrenado DenseNet121 sin la capa superior 
# (incluyendo pesos)
dn_base_model = DenseNet121(weights='imagenet', 
                            include_top=False, 
                            input_shape=input_shape)

# Agregar capas personalizadas para la clasificación
dn_x = dn_base_model.output
dn_x = GlobalAveragePooling2D()(dn_x)
dn_x = Dense(1024, activation='relu')(dn_x)
dn_predictions = Dense(total_clases, activation='softmax')(dn_x)

# Crear el modelo completo
modeloDenseNet = Model(inputs=dn_base_model.input, outputs=dn_predictions)

# Congelar las capas del modelo base (DenseNet121) 
# para no entrenarlas nuevamente
for layer in dn_base_model.layers:
    layer.trainable = False

## Compilacion de los modelos

In [55]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
#Compilado de los modelos normales
modeloV1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

modeloV2.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy', Precision(), Recall()])

modeloV3.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy', Precision(), Recall()])

modeloV4.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy', Precision(), Recall()])


In [ ]:
# Compilado de los modelos preentrenados.

# Compilado del modelo MobileNet
modeloMobileNet.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

In [56]:
# Compilado de los modelos preentrenados.

# Compilado del modelo ImageNet
modeloImageNet.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

# Compilado del modelo DenseNet
modeloDenseNet.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy', Precision(), Recall()])

## Entrenamiento de los modelos

In [57]:
from tensorflow.keras.callbacks import TensorBoard

# Calculo de la cantidad de datos a usar por epoca
steps_per_epoch_train = total_data_train // batch_size
steps_per_epoch_validation = total_data_validation // batch_size
epocas = 50

print("epocas: ", epocas)
print("Pasos por epoca para entrenar: ", steps_per_epoch_train)
print("Pasos por epoca para validar: ", steps_per_epoch_validation)

epocas:  50
Pasos por epoca para entrenar:  454
Pasos por epoca para validar:  151


In [58]:
boardModeloV1 = TensorBoard(log_dir='logs/modeloV1')

modeloV1.fit(
    train_data,
    epochs=epocas,
    batch_size=batch_size,
    validation_data=validation_data,
    steps_per_epoch=steps_per_epoch_train,
    validation_steps=steps_per_epoch_validation,
    callbacks=[boardModeloV1]
)

Epoch 1/50
454/454 [==============================] - 9s 19ms/step - loss: 1.5502 - accuracy: 0.4689 - precision_12: 0.6535 - recall_12: 0.3012 - val_loss: 1.5471 - val_accuracy: 0.5749 - val_precision_12: 0.6160 - val_recall_12: 0.5095
Epoch 2/50
454/454 [==============================] - 8s 18ms/step - loss: 0.3740 - accuracy: 0.8711 - precision_12: 0.8940 - recall_12: 0.8435 - val_loss: 1.5099 - val_accuracy: 0.6500 - val_precision_12: 0.6930 - val_recall_12: 0.6144
Epoch 3/50
454/454 [==============================] - 8s 17ms/step - loss: 0.1881 - accuracy: 0.9341 - precision_12: 0.9431 - recall_12: 0.9266 - val_loss: 1.9375 - val_accuracy: 0.6213 - val_precision_12: 0.6404 - val_recall_12: 0.6066
Epoch 4/50
454/454 [==============================] - 8s 17ms/step - loss: 0.0821 - accuracy: 0.9736 - precision_12: 0.9757 - recall_12: 0.9711 - val_loss: 1.8127 - val_accuracy: 0.6889 - val_precision_12: 0.7049 - val_recall_12: 0.6736
Epoch 5/50
454/454 [==============================] 

In [59]:
boardModeloV2 = TensorBoard(log_dir='logs/modeloV2')

modeloV2.fit(
    train_data,
    epochs=epocas, batch_size=batch_size,
    validation_data=validation_data,
    steps_per_epoch=steps_per_epoch_train,
    validation_steps=steps_per_epoch_validation,
    callbacks=[boardModeloV2]
)

Epoch 1/50
454/454 [==============================] - 8s 18ms/step - loss: 2.3167 - accuracy: 0.3956 - precision_13: 0.5733 - recall_13: 0.2034 - val_loss: 1.3390 - val_accuracy: 0.5439 - val_precision_13: 0.6111 - val_recall_13: 0.4104
Epoch 2/50
454/454 [==============================] - 8s 17ms/step - loss: 0.6019 - accuracy: 0.7773 - precision_13: 0.8247 - recall_13: 0.7121 - val_loss: 1.3811 - val_accuracy: 0.5824 - val_precision_13: 0.6353 - val_recall_13: 0.5135
Epoch 3/50
454/454 [==============================] - 8s 17ms/step - loss: 0.3517 - accuracy: 0.8741 - precision_13: 0.8956 - recall_13: 0.8470 - val_loss: 1.5733 - val_accuracy: 0.6188 - val_precision_13: 0.6570 - val_recall_13: 0.5697
Epoch 4/50
454/454 [==============================] - 8s 17ms/step - loss: 0.2028 - accuracy: 0.9284 - precision_13: 0.9388 - recall_13: 0.9188 - val_loss: 1.4986 - val_accuracy: 0.6258 - val_precision_13: 0.6526 - val_recall_13: 0.5935
Epoch 5/50
454/454 [==============================] 

In [60]:
boardModeloV3 = TensorBoard(log_dir='logs/modeloV3')

modeloV3.fit(
    train_data,
    epochs=epocas, batch_size=batch_size,
    validation_data=validation_data,
    steps_per_epoch=steps_per_epoch_train,
    validation_steps=steps_per_epoch_validation,
    callbacks=[boardModeloV3]
)

Epoch 1/50
454/454 [==============================] - 9s 19ms/step - loss: 1.5111 - accuracy: 0.4683 - precision_14: 0.6591 - recall_14: 0.3126 - val_loss: 1.4598 - val_accuracy: 0.5844 - val_precision_14: 0.6421 - val_recall_14: 0.5120
Epoch 2/50
454/454 [==============================] - 8s 18ms/step - loss: 0.4926 - accuracy: 0.8255 - precision_14: 0.8532 - recall_14: 0.7880 - val_loss: 1.5499 - val_accuracy: 0.6440 - val_precision_14: 0.6822 - val_recall_14: 0.6138
Epoch 3/50
454/454 [==============================] - 8s 18ms/step - loss: 0.2914 - accuracy: 0.8927 - precision_14: 0.9073 - recall_14: 0.8785 - val_loss: 1.5710 - val_accuracy: 0.6763 - val_precision_14: 0.7007 - val_recall_14: 0.6596
Epoch 4/50
454/454 [==============================] - 8s 18ms/step - loss: 0.1530 - accuracy: 0.9468 - precision_14: 0.9513 - recall_14: 0.9417 - val_loss: 1.9045 - val_accuracy: 0.6846 - val_precision_14: 0.7045 - val_recall_14: 0.6759
Epoch 5/50
454/454 [==============================] 

In [61]:
boardModeloV4 = TensorBoard(log_dir='logs/modeloV4')

modeloV4.fit(
    train_data,
    epochs=epocas, batch_size=batch_size,
    validation_data=validation_data,
    steps_per_epoch=steps_per_epoch_train,
    validation_steps=steps_per_epoch_validation,
    callbacks=[boardModeloV4]
)

Epoch 1/50
454/454 [==============================] - 9s 19ms/step - loss: 1.7076 - accuracy: 0.3871 - precision_15: 0.5985 - recall_15: 0.2164 - val_loss: 1.3138 - val_accuracy: 0.5710 - val_precision_15: 0.6561 - val_recall_15: 0.4489
Epoch 2/50
454/454 [==============================] - 8s 19ms/step - loss: 0.6313 - accuracy: 0.7637 - precision_15: 0.8064 - recall_15: 0.7061 - val_loss: 1.3774 - val_accuracy: 0.6405 - val_precision_15: 0.6776 - val_recall_15: 0.6072
Epoch 3/50
454/454 [==============================] - 8s 18ms/step - loss: 0.3702 - accuracy: 0.8615 - precision_15: 0.8795 - recall_15: 0.8427 - val_loss: 1.3403 - val_accuracy: 0.6486 - val_precision_15: 0.6876 - val_recall_15: 0.6095
Epoch 4/50
454/454 [==============================] - 8s 18ms/step - loss: 0.2697 - accuracy: 0.9018 - precision_15: 0.9155 - recall_15: 0.8905 - val_loss: 1.4147 - val_accuracy: 0.6815 - val_precision_15: 0.7089 - val_recall_15: 0.6633
Epoch 5/50
454/454 [==============================] 

In [ ]:
boardModeloMN = TensorBoard(log_dir='logs/ModeloMobileNet')

# Entrenamiento del modelo MobileNet
modeloMobileNet.fit(
    train_data,
    epochs=epocas, batch_size=batch_size,
    validation_data=validation_data,
    steps_per_epoch=steps_per_epoch_train,
    validation_steps=steps_per_epoch_validation,
    callbacks=[boardModeloMN]
)

In [62]:
boardModeloIN = TensorBoard(log_dir='logs/ModeloImageNet')

# Entrenamiento del modelo ImageNet
modeloImageNet.fit(
    train_data,
    epochs=epocas, batch_size=batch_size,
    validation_data=validation_data,
    steps_per_epoch=steps_per_epoch_train,
    validation_steps=steps_per_epoch_validation,
    callbacks=[boardModeloIN]
)

Epoch 1/50
454/454 [==============================] - 18s 34ms/step - loss: 1.2652 - accuracy: 0.6186 - precision_16: 0.8273 - recall_16: 0.4313 - val_loss: 1.2011 - val_accuracy: 0.6712 - val_precision_16: 0.7487 - val_recall_16: 0.6078
Epoch 2/50
454/454 [==============================] - 14s 32ms/step - loss: 0.2880 - accuracy: 0.9170 - precision_16: 0.9461 - recall_16: 0.8751 - val_loss: 1.1930 - val_accuracy: 0.6947 - val_precision_16: 0.7585 - val_recall_16: 0.6531
Epoch 3/50
454/454 [==============================] - 14s 32ms/step - loss: 0.1550 - accuracy: 0.9655 - precision_16: 0.9786 - recall_16: 0.9475 - val_loss: 1.1824 - val_accuracy: 0.7132 - val_precision_16: 0.7614 - val_recall_16: 0.6776
Epoch 4/50
454/454 [==============================] - 15s 32ms/step - loss: 0.0990 - accuracy: 0.9826 - precision_16: 0.9878 - recall_16: 0.9756 - val_loss: 1.2796 - val_accuracy: 0.7026 - val_precision_16: 0.7444 - val_recall_16: 0.6769
Epoch 5/50
454/454 [============================

In [63]:
boardModeloDN = TensorBoard(log_dir='logs/ModeloDenseNet')

# Entrenamiento del modelo DenseNet
modeloDenseNet.fit(
    train_data,
    epochs=epocas, batch_size=batch_size,
    validation_data=validation_data,
    steps_per_epoch=steps_per_epoch_train,
    validation_steps=steps_per_epoch_validation,
    callbacks=[boardModeloDN]
)

Epoch 1/50
454/454 [==============================] - 27s 54ms/step - loss: 1.0473 - accuracy: 0.7058 - precision_17: 0.8695 - recall_17: 0.5663 - val_loss: 0.6482 - val_accuracy: 0.8195 - val_precision_17: 0.8576 - val_recall_17: 0.7928
Epoch 2/50
454/454 [==============================] - 23s 50ms/step - loss: 0.0926 - accuracy: 0.9824 - precision_17: 0.9879 - recall_17: 0.9745 - val_loss: 0.6256 - val_accuracy: 0.8377 - val_precision_17: 0.8646 - val_recall_17: 0.8181
Epoch 3/50
454/454 [==============================] - 23s 50ms/step - loss: 0.0385 - accuracy: 0.9969 - precision_17: 0.9982 - recall_17: 0.9956 - val_loss: 0.6338 - val_accuracy: 0.8522 - val_precision_17: 0.8716 - val_recall_17: 0.8398
Epoch 4/50
454/454 [==============================] - 23s 50ms/step - loss: 0.0209 - accuracy: 0.9989 - precision_17: 0.9992 - recall_17: 0.9984 - val_loss: 0.6490 - val_accuracy: 0.8541 - val_precision_17: 0.8707 - val_recall_17: 0.8400
Epoch 5/50
454/454 [============================

Epoch 35/50
454/454 [==============================] - 23s 50ms/step - loss: 8.8485e-05 - accuracy: 1.0000 - precision_17: 1.0000 - recall_17: 1.0000 - val_loss: 1.3629 - val_accuracy: 0.8558 - val_precision_17: 0.8588 - val_recall_17: 0.8535
Epoch 36/50
454/454 [==============================] - 23s 50ms/step - loss: 4.3220e-05 - accuracy: 1.0000 - precision_17: 1.0000 - recall_17: 1.0000 - val_loss: 1.3610 - val_accuracy: 0.8564 - val_precision_17: 0.8592 - val_recall_17: 0.8537
Epoch 37/50
454/454 [==============================] - 23s 50ms/step - loss: 3.7367e-05 - accuracy: 1.0000 - precision_17: 1.0000 - recall_17: 1.0000 - val_loss: 1.3581 - val_accuracy: 0.8570 - val_precision_17: 0.8593 - val_recall_17: 0.8541
Epoch 38/50
454/454 [==============================] - 23s 50ms/step - loss: 2.8833e-05 - accuracy: 1.0000 - precision_17: 1.0000 - recall_17: 1.0000 - val_loss: 1.3539 - val_accuracy: 0.8568 - val_precision_17: 0.8587 - val_recall_17: 0.8543
Epoch 39/50
454/454 [=======

## Evaluacion de los modelos

In [ ]:
# Instalar en caso no se cuente con la libreria
pip install scikit-learn

In [46]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Crea un generador de datos para el conjunto de prueba
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Crea el generador de datos de prueba
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No es necesario barajar el conjunto de prueba
)

Found 159 images belonging to 16 classes.


In [49]:
# Ejecutar de hacer las evaluaciones
%run evaluador_modelos.ipynb

In [64]:
# Uso de la clase
ModelEvaluator(modeloV1, test_generator, 'modeloV1').evaluate()
ModelEvaluator(modeloV2, test_generator, 'modeloV2').evaluate()
ModelEvaluator(modeloV3, test_generator, 'modeloV3').evaluate()
ModelEvaluator(modeloV4, test_generator, 'modeloV4').evaluate()

----------Resultados del modelo: modeloV1 ------
Accuracy: 0.48427672955974843
Precision: 0.5512189458915319
Recall: 0.48427672955974843
F1 Score: 0.445589524208124
Confusion Matrix:
[[ 9  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  7  0  0  1  0  0  0  0  0  0  1  0  1  0  0]
 [ 4  0  4  1  0  0  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  6  0  0  0  1  0  0  1  0  0  0  0]
 [ 0  0  0  2  0  2  0  0  2  0  0  0  0  3  0  1]
 [ 2  0  1  1  0  0  4  1  0  0  0  0  0  1  0  0]
 [ 2  0  1  1  0  1  4  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  8  0  0  0  6  0  0  0  0  2  2  0]
 [ 0  0  0  1  0  0  0  0  2  1  0  1  0  2  2  1]
 [ 0  0  0  1  0  0  1  0  0  0  4  0  0  0  0  0]
 [ 0  0  0  0  2  0  0  0  1  0  0  5  0  1  0  1]
 [ 0  0  0  2  0  0  0  1  1  0  0  0  0  1  0  2]
 [ 0  0  0  2  0  1  0  0  0  0  0  0  0  7  0  0]
 [ 0  0  0  0  3  0  0  0  2  0  0  0  0  0  5  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  3  0  7]]


In [65]:
# El modelo MobileNet no se ejecuto por lo que no ejecutara la evaluacion
# ModelEvaluator(modeloMobileNet, test_generator, 'modeloMobileNet').evaluate()
ModelEvaluator(modeloImageNet, test_generator, 'modeloImageNet').evaluate()
ModelEvaluator(modeloDenseNet, test_generator, 'modeloDenseNet').evaluate()

----------Resultados del modelo: modeloImageNet ------
Accuracy: 0.5220125786163522
Precision: 0.5081502471439577
Recall: 0.5220125786163522
F1 Score: 0.4938978689817447
Confusion Matrix:
[[10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  5  0  0  0  0  0  0  0  1  0  0  0  0  0  0]
 [ 6  0  2  1  0  0  0  0  0  0  0  0  1  0  0  0]
 [ 1  0  0  5  1  0  1  0  1  0  1  0  0  0  0  0]
 [ 0  0  0  0  6  0  0  0  1  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  7  0  0  2  0  1  0  0  0  0  0]
 [ 2  0  0  3  1  0  3  0  1  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  1  6  0  0  0  0  1  0  1  0  0]
 [ 1  0  0  2  0  0  0  1 13  1  0  0  0  0  0  0]
 [ 0  2  0  1  2  1  1  0  2  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  4  0  0  1  0  0]
 [ 0  0  0  0  2  0  1  0  0  0  0  6  1  0  0  0]
 [ 1  0  1  0  0  0  0  1  0  0  1  0  2  0  0  1]
 [ 0  0  0  1  1  0  1  0  0  0  0  0  0  6  1  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  1  7  1]
 [ 0  0  1  0  0  0  0  0  0  0  1  0  0  0  1 

In [5]:
#Cargar la extension de tensorboard
%load_ext tensorboard

In [ ]:
from tensorboard import notebook
notebook.list() # Revisa si existen instancias de tensorboard creadas

In [7]:
#Ejecutar tensorboard e indicarle que lea la carpeta "logs"
%tensorboard --logdir logs

## Exportacion del modelo

In [8]:
modeloDenseNet.save('modeloDenseNet.h5')

NameError: name 'modeloDenseNet' is not defined